<a href="https://colab.research.google.com/github/Antika20/Movie-Recommendation-/blob/main/FLIX_MOVIE_Content_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pengumpulan dataset

In [ ]:
# Import module Untuk pengolahan data
from google.colab import files
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


# Import Module Untuk visualisasi data
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# install kaggle package
!pip install -q kaggle

In [ ]:
# upload kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"antikaorinda","key":"a0d3728340fcca4bbc22974a7fa4c54f"}'}

In [ ]:
# make directory and change permission
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [ ]:
# download dataset, choose 'copy api command' from kaggle dataset
!kaggle datasets download -d dineshaitham/movies-recommendation-dataset

  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 151MB/s]


In [ ]:
# Unzip File
!unzip /content/movies-recommendation-dataset.zip

Archive:  /content/movies-recommendation-dataset.zip
  inflating: movies.csv              
  inflating: ratings.csv             


# Data Understanding

In [ ]:
import pandas as pd

movies = pd.read_csv('/content/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
count_genre = pd.DataFrame(movies['genres'].value_counts().reset_index().values, columns = ['genre', 'count'])
print(len(count_genre))
pd.options.display.max_colwidth = 500
count_genre.head()

951


,genre,count
0,Drama,1053
1,Comedy,946
2,Comedy|Drama,435
3,Comedy|Romance,363
4,Drama|Romance,349


In [ ]:
for col in movies:
    print(f"\033[1m{col} \n{20 * '-'}\033[0m")
    print(movies[col].value_counts(), '\n')

movieId 
--------------------
1         1
53322     1
53129     1
53138     1
53140     1
         ..
4390      1
4392      1
4393      1
4394      1
193609    1
Name: movieId, Length: 9742, dtype: int64 

title 
--------------------
Emma (1996)                               2
War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Eros (2004)                               2
Saturn 3 (1980)                           2
                                         ..
Lost and Delirious (2001)                 1
Rape Me (Baise-moi) (2000)                1
Alice (1990)                              1
Another Woman (1988)                      1
Andrew Dice Clay: Dice Rules (1991)       1
Name: title, Length: 9737, dtype: int64 

genres 
--------------------
Drama                                                  1053
Comedy                                                  946
Comedy|Drama                                            435
Comedy|Romance                 

In [ ]:
# Cek missing value dengan fungsi isnull()
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

# Data Preparation

In [ ]:
# Mengurutkan movie berdasarkan movieID kemudian memasukkannya ke dalam variabel fix_movie
fix_movie = movies.sort_values('movieId', ascending=True)
fix_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
# Membuang data duplikat pada variabel preparation
preparation_movie = fix_movie.drop_duplicates('movieId')
preparation_movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
# Mengonversi data series ‘movieID’ menjadi dalam bentuk list
movie_id = preparation_movie['movieId'].tolist()

# Mengonversi data series ‘title’ menjadi dalam bentuk list
title_movie = preparation_movie['title'].tolist()

# Mengonversi data series ‘genres’ menjadi dalam bentuk list
genres_movie = preparation_movie['genres'].tolist()

print(len(movie_id))
print(len(title_movie))
print(len(genres_movie))

9742
9742
9742


In [ ]:
# Membuat dictionary untuk data ‘movie_ID’, ‘title_Movie’, dan ‘genres_Movie’
movie_new = pd.DataFrame({
    'movie_id': movie_id,
    'title': title_movie,
    'genre': genres_movie
})
movie_new

,movie_id,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data cuisine
tf.fit(movie_new['genre'])


# Mapping array dari fitur index integer  ke fitur nama
tf.get_feature_names_out()

array(['action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'fi', 'film', 'genres',
       'horror', 'imax', 'listed', 'musical', 'mystery', 'no', 'noir',
       'romance', 'sci', 'thriller', 'war', 'western'], dtype=object)

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matriks
tfidf_matrix = tf.fit_transform(movie_new['genre'])

# Melihat ukuran matriks tfidf
tfidf_matrix.shape

(9742, 24)

In [ ]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.        , 0.41684567, 0.51622547, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.51236121, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.57860574, 0.        , 0.81560738, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [ ]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan jenis get_names
# Baris diisi dengan nama title

pd.DataFrame(
    tfidf_matrix.todense(),  # title
    columns=tf.get_feature_names_out() ,
    index=movie_new.title
).sample(10, axis=1).sample(10, axis=0)

,sci,western,no,documentary,animation,comedy,fi,action,adventure,mystery
title,,,,,,,,,,
Adventures of Mowgli: The Kidnapping (1968),0.0,0.0,0.0,0.000000,0.61917,0.000000,0.0,0.000000,0.499972,0.0
Ghosts of the Abyss (2003),0.0,0.0,0.0,0.624969,0.00000,0.000000,0.0,0.000000,0.000000,0.0
Honeymoon (2014),0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0
"Breakfast Club, The (1985)",0.0,0.0,0.0,0.000000,0.00000,0.734632,0.0,0.000000,0.000000,0.0
"Fall of the House of Usher, The (House of Usher) (1960)",0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0
Half a Loaf of Kung Fu (Dian zhi gong fu gan chian chan) (1980),0.0,0.0,0.0,0.000000,0.00000,0.589966,0.0,0.807428,0.000000,0.0
Final Destination (2000),0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0
Kitchen Stories (Salmer fra kjøkkenet) (2003),0.0,0.0,0.0,0.000000,0.00000,0.734632,0.0,0.000000,0.000000,0.0
Mr. Blandings Builds His Dream House (1948),0.0,0.0,0.0,0.000000,0.00000,1.000000,0.0,0.000000,0.000000,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1.        , 0.81357774, 0.15276924, ..., 0.        , 0.4210373 ,
        0.26758648],
       [0.81357774, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15276924, 0.        , 1.        , ..., 0.        , 0.        ,
        0.57091541],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.4210373 , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.26758648, 0.        , 0.57091541, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa movie
cosine_sim_df = pd.DataFrame(cosine_sim, index=movie_new['title'], columns=movie_new['title'])
print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap movie
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (9742, 9742)


title,I Saw What You Did (1965),Spring (2015),"Perfect Storm, The (2000)","Trip to Bountiful, The (1985)","Simpsons Movie, The (2007)"
title,,,,,
Jackass Presents: Bad Grandpa (2013),0.000000,0.000000,0.000000,0.000000,0.460201
Almost Heroes (1998),0.000000,0.000000,0.000000,0.000000,0.144514
In Time (2011),0.426412,0.552007,0.351975,0.000000,0.000000
Diary of a Wimpy Kid: Dog Days (2012),0.000000,0.000000,0.000000,0.000000,0.215521
Short Circuit (1986),0.000000,0.675600,0.000000,0.000000,0.177846
Baarìa (2009),0.000000,0.000000,0.203575,0.360630,0.179701
Blue Jasmine (2013),0.000000,0.000000,0.564498,1.000000,0.000000
Anchorman 2: The Legend Continues (2013),0.000000,0.000000,0.000000,0.000000,0.460201
In the Valley of Elah (2007),0.000000,0.000000,0.240414,0.425889,0.000000


In [ ]:
def movie_recommendations(genre, similarity_data=cosine_sim_df, items=movie_new[['title', 'genre']], k=10):
    """
    Rekomendasi movie berdasarkan kemiripan dataframe

    Parameter:
    ---
    genre : tipe data string (str)
            genre (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan title sebagai
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---


    Pada index ini, kita mengambil k dengan nilai similarity terbesar
    pada index matrix yang diberikan (i).
    """


    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,genre].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Drop genre agar genre yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(genre, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
movies[movies.title.eq('The Greatest Showman (2017)')]

,movieId,title,genres
9656,180985,The Greatest Showman (2017),Drama


In [ ]:
movies[movies.genres.eq('Drama')]

,movieId,title,genres
13,14,Nixon (1995),Drama
25,26,Othello (1995),Drama
30,31,Dangerous Minds (1995),Drama
36,40,"Cry, the Beloved Country (1995)",Drama
39,43,Restoration (1995),Drama
...,...,...,...
9656,180985,The Greatest Showman (2017),Drama
9672,182793,The Purple Sea (2009),Drama
9727,190213,John From (2015),Drama
9728,190215,Liquid Truth (2017),Drama


In [ ]:
movies[movies.title.eq('Toy Story (1995)')]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
rekomendasi = pd.DataFrame(movie_recommendations('Toy Story (1995)'))
rekomendasi

,title,genre
0,The Good Dinosaur (2015),Adventure|Animation|Children|Comedy|Fantasy
1,Moana (2016),Adventure|Animation|Children|Comedy|Fantasy
2,"Adventures of Rocky and Bullwinkle, The (2000)",Adventure|Animation|Children|Comedy|Fantasy
3,"Wild, The (2006)",Adventure|Animation|Children|Comedy|Fantasy
4,"Emperor's New Groove, The (2000)",Adventure|Animation|Children|Comedy|Fantasy
5,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy
6,Turbo (2013),Adventure|Animation|Children|Comedy|Fantasy
7,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
8,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy
9,Asterix and the Vikings (Astérix et les Vikings) (2006),Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
value = pd.DataFrame(rekomendasi['genre'].value_counts().reset_index().values, columns = ['genre', 'count'])
value.head()

,genre,count
0,Adventure|Animation|Children|Comedy|Fantasy,10


In [ ]:
rekomendasi = pd.DataFrame(movie_recommendations('The Greatest Showman (2017)'))
rekomendasi

,title,genre
0,Monsieur Ibrahim (Monsieur Ibrahim et les fleurs du Coran) (2003),Drama
1,Proof (2005),Drama
2,Miss Meadows (2014),Drama
3,Melvin and Howard (1980),Drama
4,Bringing Out the Dead (1999),Drama
5,Body Shots (1999),Drama
6,Songs From the Second Floor (Sånger från andra våningen) (2000),Drama
7,If These Walls Could Talk (1996),Drama
8,Radio Flyer (1992),Drama
9,Get on the Bus (1996),Drama


In [ ]:
value = pd.DataFrame(rekomendasi['genre'].value_counts().reset_index().values, columns = ['genre', 'count'])
value.head()

,genre,count
0,Drama,10


# Evaluasi

In [ ]:
TP = 10 #jumlah prediksi benar untuk genre/ title yang mirip atau serupa
FP = 0 #jumlah prediksi salah untuk genre/title yang mirip atau serupa

Precision = TP/(TP+FP)
print("{0:.0%}".format(Precision))

100%


In [ ]:
TP = 10 #jumlah prediksi benar untuk genre yang mirip atau serupa
FN = 0 #jumlah data yang benar diklasifikasikan sebagai kelas negatif

Recall = TP/(TP+FN)
print("{0:.0%}".format(Recall))

100%


In [ ]:
#F1-SCORE

F1 = 2* Precision * Recall / (Precision  + Recall )
print("{0:.0%}".format(F1))

100%


In [ ]:
# Accuracy
TP = 10 # jumlah data yang benar diklasifikasikan sebagai kelas positif
TN = 0 # jumlah data yang salah diklasifikasikan sebagai kelas positif
FN = 0 # jumlah data yang benar diklasifikasikan sebagai kelas negatif
FP = 0 # jumlah data yang salah diklasifikasikan sebagai kelas negatif

Accuracy = (TP +TN) / (TP+TN+FP+FN)
print("{0:.0%}".format(Accuracy))


100%
